### Modules and configurations

In [6]:
import os

from dotenv import load_dotenv

from azure.cosmos import CosmosClient
from azure.cosmos.partition_key import PartitionKey

In [2]:
load_dotenv("env/.secrets.env")

COSMOS_DB_URL = os.getenv("COSMOS_DB_URL")
ACCOUNT_KEY = os.getenv("ACCOUNT_KEY")

In [4]:
cosmos_client = CosmosClient(url=COSMOS_DB_URL, credential=ACCOUNT_KEY)

### Create Database and Container

In [7]:
database_client = cosmos_client.create_database_if_not_exists("mdb637")
container_client = database_client.create_container_if_not_exists(
    id="users",
    partition_key=PartitionKey(path="/userId"),
)

### Create item (s)

In [8]:
users = [
    {
        "id": "1",
        "userId": "u111",
        "name": "Maksym",
    },
    {
        "id": "2",
        "userId": "u222",
        "name": "Bohdan",
    },
]

for user in users:
    container_client.create_item(user)

### Read item (s)

In [10]:
for item, user_id in [("1", "u111"), ("2", "u222")]:
    print(container_client.read_item(item=item, partition_key=user_id))

{'id': '1', 'userId': 'u111', 'name': 'Maksym', '_rid': 'HdVTAPMzQJEBAAAAAAAAAA==', '_self': 'dbs/HdVTAA==/colls/HdVTAPMzQJE=/docs/HdVTAPMzQJEBAAAAAAAAAA==/', '_etag': '"c9023dcc-0000-0c00-0000-6928d81e0000"', '_attachments': 'attachments/', '_ts': 1764284446}
{'id': '2', 'userId': 'u222', 'name': 'Bohdan', '_rid': 'HdVTAPMzQJECAAAAAAAAAA==', '_self': 'dbs/HdVTAA==/colls/HdVTAPMzQJE=/docs/HdVTAPMzQJECAAAAAAAAAA==/', '_etag': '"c9023ecc-0000-0c00-0000-6928d81e0000"', '_attachments': 'attachments/', '_ts': 1764284446}


### Query item (s)

In [14]:
items = container_client.query_items(
    query="SELECT c.name FROM c WHERE c.userId = @uid",
    parameters=[{"name": "@uid", "value": "u111"}],
    enable_cross_partition_query=True,
)

for item in items:
    print(item)

{'name': 'Maksym'}


### Delete item (s)

In [15]:
container_client.delete_item(item="1", partition_key="u111")